In [29]:
!nvidia-smi


Thu Oct  2 07:38:32 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [32]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [35]:
%%bash
cat > cuda_vector_demo.cu <<'EOF'
#include <cstdio>
#include <cstdlib>
#include <cuda_runtime.h>

#define N 1024

#define CHECK(call) { \
    cudaError_t e = (call); \
    if (e != cudaSuccess) { \
        fprintf(stderr, "CUDA Error: %s (file %s, line %d)\n", cudaGetErrorString(e), __FILE__, __LINE__); \
        exit(1); \
    } \
}

// ========================== KERNELS =============================

__global__ void kernel_serial(int *A, int *B, int *C, int *D) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < N) {
        A[i] = i;
        B[i] = 2*i;
        C[i] = i*i;
        D[i] = A[i] + B[i] + C[i]; // expect 9*i^2
    }
}

__global__ void kernel_partition(int *A, int *B, int *C, int *D, int offset) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int gi = offset + i;
    if (gi < N) {
        A[gi] = gi;
        B[gi] = 2*gi;
        C[gi] = gi*gi;
        D[gi] = A[gi] + B[gi] + C[gi];
    }
}

__global__ void kernel_badRace(int *A, int *B, int *C, int *D) {
    int i = threadIdx.x;
    A[0] = i;
    B[0] = 2*i;
    C[0] = i*i;
    D[0] = A[0] + B[0] + C[0];
}

__global__ void kernel_goodRace(int *A, int *B, int *C, int *D) {
    int i = threadIdx.x;
    __shared__ int tmpA, tmpB, tmpC;
    if (i == 0) {
        tmpA = 0; tmpB = 0; tmpC = 0;
    }
    __syncthreads();
    atomicAdd(&tmpA, i);
    atomicAdd(&tmpB, 2*i);
    atomicAdd(&tmpC, i*i);
    __syncthreads();
    if (i == 0) {
        D[0] = tmpA + tmpB + tmpC;
    }
}

__global__ void kernel_map(int *out) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    out[i] = i;
}

// ============== Reduction (Fixed for CUDA 12.5) ==================

__global__ void reductionKernel(int *D, unsigned long long *sum) {
    __shared__ int temp[1024];
    int tid = threadIdx.x;
    int i = blockIdx.x * blockDim.x + tid;

    temp[tid] = D[i];
    __syncthreads();

    for (int stride = blockDim.x/2; stride > 0; stride >>= 1) {
        if (tid < stride) {
            temp[tid] += temp[tid + stride];
        }
        __syncthreads();
    }

    if (tid == 0) {
        atomicAdd(sum, (unsigned long long)temp[0]); // FIXED
    }
}

// ========================== MAIN =============================

int main() {
    printf("CUDA Vector Ops Demo (N=%d)\n\n", N);

    int *dA, *dB, *dC, *dD;
    CHECK(cudaMalloc(&dA, N*sizeof(int)));
    CHECK(cudaMalloc(&dB, N*sizeof(int)));
    CHECK(cudaMalloc(&dC, N*sizeof(int)));
    CHECK(cudaMalloc(&dD, N*sizeof(int)));

    // ===== Demo 1: Serial execution =====
    printf("=== Demo 1: Serial execution on default stream ===\n");
    kernel_serial<<<N/256, 256>>>(dA, dB, dC, dD);
    CHECK(cudaDeviceSynchronize());

    int hD[8];
    CHECK(cudaMemcpy(hD, dD, 8*sizeof(int), cudaMemcpyDeviceToHost));
    for (int i=0; i<8; i++) {
        printf("D[%d] = %d\n", i, hD[i]);
    }
    printf("\n");

    // ===== Demo 2: Streams + partitioning =====
    printf("=== Demo 2: Streams + partitioning ===\n");
    cudaStream_t s1, s2;
    cudaStreamCreate(&s1);
    cudaStreamCreate(&s2);

    kernel_partition<<<N/512, 256, 0, s1>>>(dA, dB, dC, dD, 0);
    kernel_partition<<<N/512, 256, 0, s2>>>(dA, dB, dC, dD, N/2);
    CHECK(cudaDeviceSynchronize());

    int hD2[8];
    CHECK(cudaMemcpy(hD2, dD, 4*sizeof(int), cudaMemcpyDeviceToHost));
    CHECK(cudaMemcpy(hD2+4, dD+512, 4*sizeof(int), cudaMemcpyDeviceToHost));
    for (int i=0; i<8; i++) {
        printf("D[%d] = %d\n", (i<4)?i:512+(i-4), hD2[i]);
    }
    printf("\n");

    // ===== Demo 3: Race condition =====
    printf("=== Demo 3: Race condition + fix ===\n");
    kernel_badRace<<<1, 256>>>(dA, dB, dC, dD);
    CHECK(cudaDeviceSynchronize());
    CHECK(cudaMemcpy(hD, dD, 5*sizeof(int), cudaMemcpyDeviceToHost));
    printf("Bad (racy) results sample:\n");
    for (int i=0; i<5; i++) printf("D[%d] = %d\n", i, hD[i]);

    kernel_goodRace<<<1, 256>>>(dA, dB, dC, dD);
    CHECK(cudaDeviceSynchronize());
    CHECK(cudaMemcpy(hD, dD, 5*sizeof(int), cudaMemcpyDeviceToHost));
    printf("Fixed (event-ordered) results sample:\n");
    for (int i=0; i<5; i++) printf("D[%d] = %d\n", i, hD[i]);
    printf("\n");

    // ===== Demo 4: cudaMemcpyAsync =====
    printf("=== Demo 4: cudaMemcpyAsync + stream sync ===\n");
    int *hD_pinned;
    CHECK(cudaMallocHost(&hD_pinned, 5*sizeof(int)));
    for (int i=0; i<5; i++) hD_pinned[i] = -999;

    cudaMemcpyAsync(hD_pinned, dD, 5*sizeof(int), cudaMemcpyDeviceToHost, s1);
    printf("Without synchronization: may print stale values:\n");
    for (int i=0; i<5; i++) printf("hD_pinned[%d] = %d\n", i, hD_pinned[i]);

    cudaStreamSynchronize(s1);
    printf("After cudaStreamSynchronize (correct):\n");
    for (int i=0; i<5; i++) printf("hD_pinned[%d] = %d\n", i, hD_pinned[i]);
    printf("\n");

    // ===== Demo 5: Thread hierarchy mapping =====
    printf("=== Demo 5: Thread hierarchy mapping ===\n");
    int *dOut, *hOut;
    CHECK(cudaMalloc(&dOut, N*sizeof(int)));
    hOut = (int*)malloc(N*sizeof(int));

    kernel_map<<<1, N>>>(dOut);
    CHECK(cudaMemcpy(hOut, dOut, 8*sizeof(int), cudaMemcpyDeviceToHost));
    printf("Mapping for <<<1, N>>> sample:\n");
    for (int i=0; i<8; i++) printf("out[%d] = %d\n", i, hOut[i]);

    kernel_map<<<N/32, 32>>>(dOut);
    CHECK(cudaMemcpy(hOut, dOut, 8*sizeof(int), cudaMemcpyDeviceToHost));
    printf("Mapping for <<<N/32, 32>>> sample:\n");
    for (int i=0; i<8; i++) {
        printf("global index %d -> blockIdx=%d threadIdx=%d out[%d]=%d\n",
               i, i/32, i%32, i, hOut[i]);
    }
    printf("\n");

    // ===== Bonus: reduction =====
    printf("=== Bonus: reduction (shared + atomic / partials) ===\n");
    unsigned long long *dSum;
    CHECK(cudaMalloc(&dSum, sizeof(unsigned long long)));
    CHECK(cudaMemset(dSum, 0, sizeof(unsigned long long)));

    reductionKernel<<<N/1024, 1024>>>(dD, dSum);
    CHECK(cudaDeviceSynchronize());

    unsigned long long hSum;
    CHECK(cudaMemcpy(&hSum, dSum, sizeof(unsigned long long), cudaMemcpyDeviceToHost));
    printf("Sum via shared-block + atomicAdd: %llu\n", hSum);

    long long cpuSum = 0;
    int *hAll = (int*)malloc(N*sizeof(int));
    CHECK(cudaMemcpy(hAll, dD, N*sizeof(int), cudaMemcpyDeviceToHost));
    for (int i=0; i<N; i++) cpuSum += hAll[i];
    printf("CPU direct sum of D[]: %lld\n", cpuSum);

    printf("\nAll demos completed.\n");
    return 0;
}
EOF


In [36]:
%%bash
nvcc -arch=sm_75 -O2 cuda_vector_demo.cu -o demo
./demo


CUDA Vector Ops Demo (N=1024)

=== Demo 1: Serial execution on default stream ===
D[0] = 0
D[1] = 4
D[2] = 10
D[3] = 18
D[4] = 28
D[5] = 40
D[6] = 54
D[7] = 70

=== Demo 2: Streams + partitioning ===
D[0] = 0
D[1] = 4
D[2] = 10
D[3] = 18
D[512] = 263680
D[513] = 264708
D[514] = 265738
D[515] = 266770

=== Demo 3: Race condition + fix ===
Bad (racy) results sample:
D[0] = 9504
D[1] = 4
D[2] = 10
D[3] = 18
D[4] = 28
Fixed (event-ordered) results sample:
D[0] = 5657600
D[1] = 4
D[2] = 10
D[3] = 18
D[4] = 28

=== Demo 4: cudaMemcpyAsync + stream sync ===
Without synchronization: may print stale values:
hD_pinned[0] = -999
hD_pinned[1] = -999
hD_pinned[2] = -999
hD_pinned[3] = -999
hD_pinned[4] = -999
After cudaStreamSynchronize (correct):
hD_pinned[0] = 5657600
hD_pinned[1] = 4
hD_pinned[2] = 10
hD_pinned[3] = 18
hD_pinned[4] = 28

=== Demo 5: Thread hierarchy mapping ===
Mapping for <<<1, N>>> sample:
out[0] = 0
out[1] = 1
out[2] = 2
out[3] = 3
out[4] = 4
out[5] = 5
out[6] = 6
out[7] = 7
